
# 🦾 Biomech Research: Report & Share — Colab Template

**What this is:** a lean template you can copy, run, and tweak. It walks you from **project info → data → parse/clean → analyze → visualize → compose report → export (MD/HTML/PNG/PDF)**.

- Beginner-friendly. Advanced-friendly. Minimal moving parts.
- Every step is optional; swap in your own workflow as needed.
- Use the **demo data** to learn, or **upload your own** CSV.
- Exports are saved to `/content` on Colab (or current working dir).

> Tip: Run cells top-to-bottom the first time. Then edit.

poseiq.com | Dr. Hossein Mokhtarzadeh


In [ ]:

# --- Setup (libraries) ---
# Keep it light. Only install optional PDF tool if you want native PDF from HTML.
# If you're on Colab, internet is available; otherwise this will be skipped safely.

import sys, os, math, io, datetime, json, textwrap, base64
from dataclasses import dataclass
from typing import Optional, Dict, Any, List

# Core libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: HTML→PDF (best-effort). If install fails, we keep going happily.
try:
    import subprocess, shutil
    if shutil.which("pip"):
        subprocess.run([sys.executable, "-m", "pip", "install", "xhtml2pdf", "-q"], check=False)
    from xhtml2pdf import pisa  # type: ignore
    _PDF_OK = True
except Exception:
    _PDF_OK = False

print("PDF export available:", _PDF_OK)
print("Ready.")


## 1) Project Info — fill these and run

In [ ]:

# Minimal project meta. Edit and re-run anytime.
project = {
    "title": "Landing Mechanics and Fatigue",
    "authors": "A. Researcher, B. Scientist",
    "affiliation": "Human Performance Lab",
    "date": str(datetime.date.today()),
    "objective": "Quantify changes in lower-limb joint kinematics during single-leg drop landing under fatigue.",
    "notes": "Replace demo with your own data when ready."
}

# Short checklist to keep focus
checklist = [
    "Define the question clearly",
    "Verify data quality (units, gaps, outliers)",
    "Keep plots clean and labeled",
    "Prefer simple stats before complex models",
    "Write for a human, not just a method section"
]

project


## 2) Data — upload a CSV or generate demo

In [ ]:

# Option A: Upload your CSV (columns example: time, knee_pre, knee_post)
# - In Colab: uncomment below to use the uploader
# from google.colab import files
# up = files.upload()   # pick your CSV
# import io
# df = pd.read_csv(io.BytesIO(list(up.values())[0]))

# Option B: Demo data (safe default)
time = np.linspace(0, 100, 201)  # % stance (0-100)
knee_pre  = 60 + 30*np.sin(np.pi*time/100)
knee_post = 60 + 25*np.sin(np.pi*time/100)

# Add small noise for realism
rng = np.random.default_rng(42)
knee_pre  = knee_pre  + rng.normal(0, 0.6, size=time.size)
knee_post = knee_post + rng.normal(0, 0.6, size=time.size)

df = pd.DataFrame({"time": time, "knee_pre": knee_pre, "knee_post": knee_post})
df.head()


## 3) Parse & Clean — keep this light, add your rules

In [ ]:

# Example cleaning steps. Adjust as needed.
# 1) Drop obvious duplicates
df = df.drop_duplicates(subset=["time"])

# 2) Sort by time
df = df.sort_values("time").reset_index(drop=True)

# 3) Clip out-of-range knee angles (example sanity check)
df["knee_pre"]  = df["knee_pre"].clip(0, 180)
df["knee_post"] = df["knee_post"].clip(0, 180)

# 4) (Optional) Smooth a little with rolling mean (comment out if not desired)
df["knee_pre_s"]  = df["knee_pre"].rolling(5, center=True, min_periods=1).mean()
df["knee_post_s"] = df["knee_post"].rolling(5, center=True, min_periods=1).mean()

df.head()


## 4) Analyze — simple stats first

In [ ]:

# Example: Peak flexion and area-under-curve difference
def peak_and_auc(y, x):
    # x should be increasing. AUC via trapezoid.
    peak = float(np.max(y))
    auc  = float(np.trapz(y, x))
    return peak, auc

pre_peak,  pre_auc  = peak_and_auc(df["knee_pre_s"].values,  df["time"].values)
post_peak, post_auc = peak_and_auc(df["knee_post_s"].values, df["time"].values)

delta_peak = post_peak - pre_peak
delta_auc  = post_auc - pre_auc

analysis = {
    "pre_peak": round(pre_peak, 2),
    "post_peak": round(post_peak, 2),
    "delta_peak": round(delta_peak, 2),
    "pre_auc": round(pre_auc, 2),
    "post_auc": round(post_auc, 2),
    "delta_auc": round(delta_auc, 2),
}

analysis


## 5) Visualize — clean, single plots

In [ ]:

# Plot 1: Knee angle waveforms
plt.figure(figsize=(6,4))
plt.plot(df["time"], df["knee_pre_s"], label="Pre-Fatigue")
plt.plot(df["time"], df["knee_post_s"], label="Post-Fatigue", linestyle="--")
plt.xlabel("% Stance Phase")
plt.ylabel("Knee Angle (°)")
plt.title("Knee Flexion During Landing")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# (Optional) Add more plots in separate cells, one per figure.


## 6) Compose Report — Markdown + auto-fill fields

In [ ]:

# Build a simple Markdown report. You can edit the template string.
report_md = f"""
# 🦾 Biomechanics Research Brief: *{project['title']}*

**Authors:** {project['authors']}
**Affiliation:** {project['affiliation']}
**Date:** {project['date']}

---

## 🎯 Objective
{project['objective']}

---

## ⚙️ Methods Summary (example)
- Participants: n = 10 trained athletes (25 ± 4 y)  <!-- EDIT -->
- Task: Single-leg drop landing from 40 cm           <!-- EDIT -->
- Data: 3D motion (200 Hz), force plate (1000 Hz)    <!-- EDIT -->
- Processing: Butterworth filter (6 Hz)               <!-- EDIT -->
- Variables: Joint angles, GRF, joint moments        <!-- EDIT -->
- Analysis: Peak values, waveform comparison         <!-- EDIT -->

---

## 📊 Key Results (example)
- Peak Knee Flexion (°): Pre = {analysis['pre_peak']}  | Post = {analysis['post_peak']}  | Δ = {analysis['delta_peak']}
- Knee Angle AUC: Pre = {analysis['pre_auc']} | Post = {analysis['post_auc']} | Δ = {analysis['delta_auc']}

**Summary:** Replace with your interpretation and practical implications.

---

## 🖼️ Plots
Add exported figures here (saved separately as PNG).

---

## ✅ Checklist
""" + "\n".join([f"- [ ] {item}" for item in checklist]) + """

---

*Generated with Python • Matplotlib • Pandas.*
"""

# Save Markdown
md_path = "biomech_report.md"
with open(md_path, "w", encoding="utf-8") as f:
    f.write(report_md)

print("Markdown saved to", md_path)
print(report_md[:500] + " ...")


## 7) Export — HTML / PNG summary / (optional) PDF

In [ ]:

# Helper functions to export the report in multiple formats.
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

def export_html(md_text: str, out_html="biomech_report.html"):
    # Minimal HTML wrapper; you can add CSS as needed.
    style = """
    <style>
    body{font-family: Arial, sans-serif; max-width: 820px; margin: 2rem auto; line-height:1.6;}
    h1,h2,h3{margin-top:1.2em}
    code, pre{background:#f6f8fa; padding: 2px 4px; border-radius:4px}
    hr{border:none; border-top:1px solid #ddd; margin:1.5rem 0}
    </style>
    """
    # For robustness and zero extra deps, we wrap the Markdown as <pre>.
    final = f"<!DOCTYPE html><html><head><meta charset='utf-8'>{style}</head><body><pre>{md_text}</pre></body></html>"
    with open(out_html, "w", encoding="utf-8") as f:
        f.write(final)
    return out_html

def export_png_summary(md_text: str, figure_path: Optional[str] = None, out_png="biomech_report.png"):
    # Render a clean PNG with a short text summary + optional figure pasted below.
    W, M = 1200, 50
    # Try fonts
    font_paths = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
        "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/SFNS.ttf",
    ]
    def load_font(size):
        for p in font_paths:
            if os.path.exists(p):
                try:
                    return ImageFont.truetype(p, size=size)
                except Exception:
                    pass
        return ImageFont.load_default()

    title_font = load_font(48)
    body_font  = load_font(24)

    # Build a compact summary by grabbing key sections
    lines = md_text.splitlines()
    keep_blocks = []
    cur = []
    capturing = False
    for ln in lines:
        if ln.startswith("# 🦾 Biomechanics Research Brief:"):
            capturing = True
        if capturing:
            cur.append(ln)
        if ln.strip() == "---" and capturing and len(cur) > 0:
            keep_blocks.extend(cur[:30])  # keep a chunk
            break
    short_text = "\n".join(keep_blocks) if keep_blocks else md_text[:1200]

    dummy = Image.new("RGB", (W, 100), "white")
    d = ImageDraw.Draw(dummy)
    def wrapped_lines(text, font, max_width=W-2*M):
        words = text.split()
        line = ""
        out = []
        for w in words:
            trial = (line + " " + w).strip()
            if d.textlength(trial, font=font) <= max_width:
                line = trial
            else:
                out.append(line)
                line = w
        if line:
            out.append(line)
        return out

    header = "🦾 Biomechanics Research Brief"
    header_h = int(title_font.size * 1.4)
    body_lines = wrapped_lines(short_text, body_font)
    body_h = int(len(body_lines) * (body_font.size * 1.3))

    fig_h = 0
    fig_img = None
    if figure_path and os.path.exists(figure_path):
        fig_img = Image.open(figure_path).convert("RGB")
        ratio = fig_img.height / fig_img.width
        fig_img = fig_img.resize((W - 2*M, int((W - 2*M) * ratio)))
        fig_h = fig_img.height + 30

    H = M + header_h + 10 + body_h + 20 + fig_h + M
    img = Image.new("RGB", (W, H), "white")
    draw = ImageDraw.Draw(img)

    draw.text((M, M), header, font=title_font, fill=(0,0,0))
    y = M + header_h + 10

    for line in body_lines:
        draw.text((M, y), line, font=body_font, fill=(0,0,0))
        y += int(body_font.size * 1.3)

    y += 20
    if fig_img:
        img.paste(fig_img, (M, y))
        y += fig_img.height + 10

    img.save(out_png, dpi=(300,300))
    return out_png

def export_pdf_from_html(html_path: str, out_pdf="biomech_report.pdf"):
    if not os.path.exists(html_path):
        raise FileNotFoundError(html_path)
    try:
        from xhtml2pdf import pisa  # type: ignore
    except Exception:
        raise RuntimeError("PDF engine not available. Use 'File > Print > Save as PDF' on the HTML instead.")
    with open(html_path, "r", encoding="utf-8") as f:
        html = f.read()
    with open(out_pdf, "wb") as f:
        pisa.CreatePDF(html, dest=f)  # type: ignore
    return out_pdf

print("Export helpers ready. Use the next cell to export.")


In [ ]:

# --- Export now ---
# 1) Save plots you want to include in the PNG summary
plt.figure(figsize=(6,4))
plt.plot(df["time"], df["knee_pre_s"], label="Pre-Fatigue")
plt.plot(df["time"], df["knee_post_s"], label="Post-Fatigue", linestyle="--")
plt.xlabel("% Stance Phase")
plt.ylabel("Knee Angle (°)")
plt.title("Knee Flexion During Landing")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plot_path = "knee_plot.png"
plt.savefig(plot_path, dpi=300)
plt.close()

# 2) Load markdown we wrote earlier
with open("biomech_report.md", "r", encoding="utf-8") as f:
    md_text = f.read()

# 3) Export formats
html_path = export_html(md_text, out_html="biomech_report.html")
png_path  = export_png_summary(md_text, figure_path=plot_path, out_png="biomech_report.png")

# 4) Optional: PDF (best-effort). If the engine isn't available, just open the HTML and print to PDF.
pdf_path = None
try:
    pdf_path = export_pdf_from_html(html_path, out_pdf="biomech_report.pdf")
except Exception as e:
    print("PDF export fallback:", e)
    print("Use the HTML file -> print to PDF instead.")

print("\nFiles created:")
print(" -", "biomech_report.md")
print(" -", html_path)
print(" -", png_path)
if pdf_path:
    print(" -", pdf_path)



### (Optional) Save to Google Drive
Uncomment and run to mount your Drive and copy outputs.


In [ ]:

# from google.colab import drive
# drive.mount('/content/drive')
# !mkdir -p "/content/drive/MyDrive/biomech_reports"
# !cp biomech_report.* knee_plot.png "/content/drive/MyDrive/biomech_reports/"
# print("Copied to /content/drive/MyDrive/biomech_reports/")


In [ ]:
# !pip install python-docx
# from docx import Document

# doc = Document()
# for line in report_md.splitlines():
#     doc.add_paragraph(line)
# doc.save("biomech_report.docx")
# print("Saved biomech_report.docx ✅")
